In [ ]:
import numpy as np
import pandas as pd 
import astropy.io.ascii as ascii
from astropy.table import Table
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
import PIL
from PIL import Image
import sys
import os
import matplotlib.image as mp
from astropy.utils.data import download_file
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib.image as mp
from astropy.visualization import LinearStretch
from astropy.visualization import ZScaleInterval
from astropy.visualization import ImageNormalize

# Functions needed for the matching process

In [ ]:
def match1(poi, catalogue):
    
    x=poi['X_WORLD']
    y=poi['Y_WORLD']

    coord1 = SkyCoord(ra=[x]*u.degree, dec=[y]*u.degree)
    coord2 = SkyCoord(ra=catalogue['X_WORLD']*u.degree, dec=catalogue['Y_WORLD']*u.degree)

    idx, d2d, d3d = coord1.match_to_catalog_sky(coord2)
    max_sep = 1.0 * u.arcsec
    sep_constraint = d2d < max_sep
    global catalogue_match
    catalogue_match = catalogue.iloc[idx[sep_constraint]]
    
    
def match2(catalogue1, catalogue2):
    xworld1 = catalogue1['X_WORLD']
    yworld1 = catalogue1['Y_WORLD']

    xworld2 = catalogue2['X_WORLD']
    yworld2 = catalogue2['Y_WORLD']

    coord1 = SkyCoord(ra=xworld1*u.degree, dec=yworld1*u.degree)
    coord2 = SkyCoord(ra=xworld2*u.degree, dec=yworld2*u.degree)

    idx, d2d, d3d = coord1.match_to_catalog_sky(coord2)

    max_sep = 1.0 * u.arcsec

    sep_constraint = d2d < max_sep

    global data1_12_matches
    try: data1_12_matches = catalogue1[sep_constraint]
    except IndexError: 
        pass 
    
    global data2_12_matches
    data2_12_matches = catalogue2.iloc[idx[sep_constraint]]

    
def match3(catalogue1, catalogue2, catalogue3):
    xworld1 = catalogue1['X_WORLD']
    yworld1 = catalogue1['Y_WORLD']

    xworld2 = catalogue2['X_WORLD']
    yworld2 = catalogue2['Y_WORLD']
    
    xworld3 = catalogue3['X_WORLD']
    yworld3 = catalogue3['Y_WORLD']
    
    coord1 = SkyCoord(ra=xworld1*u.degree, dec=yworld1*u.degree)
    coord2 = SkyCoord(ra=xworld2*u.degree, dec=yworld2*u.degree)

    
    idx, d2d, d3d = coord1.match_to_catalog_sky(coord2)
    
    max_sep = 1.0 * u.arcsec

    sep_constraint = d2d < max_sep
    
    data1_12_matches = catalogue1[sep_constraint]
    data2_12_matches = catalogue2.iloc[idx[sep_constraint]]
    
    xworld1_12 = data1_12_matches['X_WORLD']
    yworld1_12 = data1_12_matches['Y_WORLD']
    
    coord1_12 = SkyCoord(ra=xworld1_12*u.degree, dec=yworld1_12*u.degree)
    coord3 = SkyCoord(ra=xworld3*u.degree, dec=yworld3*u.degree)
    
    idx, d2d, d3d = coord1_12.match_to_catalog_sky(coord3)
    
    max_sep = 1.0 * u.arcsec

    sep_constraint = d2d < max_sep
    
    global data1_123_matches
    data1_123_matches = data1_12_matches[sep_constraint]
    global data2_123_matches
    data2_123_matches = data2_12_matches[sep_constraint]
    global data3_123_matches
    data3_123_matches = catalogue3.iloc[idx[sep_constraint]]

# This is for the inputs when typing in the terminal. Make sure that these are the subtraction images and g data should be first for the light curve labels to be correct

In [ ]:
nargs = len(sys.argv) - 1
if nargs == 12: 
    df1 = pd.read_csv(sys.argv[1])
    df2 = pd.read_csv(sys.argv[2])
    df3 = pd.read_csv(sys.argv[3])
    df4 = pd.read_csv(sys.argv[4])
    df5 = pd.read_csv(sys.argv[5])
    df6 = pd.read_csv(sys.argv[6])
    r7 = ascii.read(sys.argv[7])
    r8 = ascii.read(sys.argv[8])
    r9 = ascii.read(sys.argv[9])
    r10 = ascii.read(sys.argv[10])
    r11 = ascii.read(sys.argv[11])
    r12 = ascii.read(sys.argv[12])

else:
    print("Error: Arguments invalid")
    print('Example: python Make_Lightcurves.py g1_SUB.cat g2_SUB.cat g3_SUB.cat i1_SUB.cat i2_SUB.cat i3_SUB.cat' +
          'g1_SCI.cat g2_SCI.cat g3_SCI.cat i1_SCI.cat i2_SCI.cat i3_SCI.cat')
    sys.exit()

In [ ]:
#reads in the cat files as panda dataframes
df7 = pd.DataFrame(r7.as_array())
df8 = pd.DataFrame(r8.as_array())
df9 = pd.DataFrame(r9.as_array())
df10 = pd.DataFrame(r10.as_array())
df11 = pd.DataFrame(r11.as_array())
df12 = pd.DataFrame(r12.as_array())

print('data points in first file:', len(df1))
print('data points in second file:', len(df2))
print('data points in third file:', len(df3))
print('data points in fourth file:', len(df4))
print('data points in fifth file:', len(df5))
print('data points in sixth file:', len(df6))

print('data points in seventh file:', len(r7))
print('data points in eighth file:', len(r8))
print('data points in ninth file:', len(r9))
print('data points in tenth file:', len(r10))
print('data points in eleventh file:', len(r11))
print('data points in twelfth file:', len(r12))

#getting rid of the points in the sub images with a mag_auto of greater than 20 as these are errors
# df1f = df1[df1['MAG_AUTO'] < 20]
# df2f = df2[df2['MAG_AUTO'] < 20]
# df3f = df3[df3['MAG_AUTO'] < 20] 
# df4f = df4[df4['MAG_AUTO'] < 20]
# df5f = df5[df5['MAG_AUTO'] < 20]
# df6f = df6[df6['MAG_AUTO'] < 20]
df1f = df1.drop(columns = 'Unnamed: 0')
df2f = df2.drop(columns = 'Unnamed: 0')
df3f = df3.drop(columns = 'Unnamed: 0')
df4f = df4.drop(columns = 'Unnamed: 0')
df5f = df5.drop(columns = 'Unnamed: 0')
df6f = df6.drop(columns = 'Unnamed: 0')
df7f = df7[df7['MAG_AUTO'] < 25]
df8f = df8[df8['MAG_AUTO'] < 25]
df9f = df9[df9['MAG_AUTO'] < 25]
df10f = df10[df10['MAG_AUTO'] < 25]
df11f = df11[df11['MAG_AUTO'] < 25]
df12f = df12[df12['MAG_AUTO'] < 25]


print('valid data points in first file:', len(df1f))
print('valid data points in second file:', len(df2f))
print('valid data points in third file:', len(df3f))
print('valid data points in fourth file:', len(df4f))
print('valid data points in fifth file:', len(df5f))
print('valid data points in sixth file:', len(df6f))

print('valid data points in seventh file:', len(df7f))
print('valid data points in eighth file:', len(df8f))
print('valid data points in ninth file:', len(df9f))
print('valid data points in tenth file:', len(df10f))
print('valid data points in eleventh file:', len(df11f))
print('valid data points in twelfth file:', len(df12f))

# The code below is for the matching process and making the lightcurves

In [ ]:
match3(df1f, df2f, df3f) #finding common sources over the 3 inputs
print('There are', len(data1_123_matches), 'matches between the first 3 files')
plt.rcParams.update({'figure.max_open_warning': 0})

# Making the panda dataframes needed for exporting all of the data
mag_csv = [] #dataframe for exporting to csv 
gsub1 = []
gsub2 = []
gsub3 = []
isub1 = []
isub2 = []
isub3 = []
gsci1 = []
gsci2 = []
gsci3 = []
isci1 = []
isci2 = []
isci3 = []

xgsub1 = []
xgsub2 = []
xgsub3 = [] 
xisub1 = [] 
xisub2 = []
xisub3 = []
xgsci1 = []
xgsci2 = []
xgsci3 = [] 
xisci1 = [] 
xisci2 = []
xisci3 = []

ygsub1 = []
ygsub2 = []
ygsub3 = [] 
yisub1 = [] 
yisub2 = []
yisub3 = []
ygsci1 = []
ygsci2 = []
ygsci3 = [] 
yisci1 = [] 
yisci2 = []
yisci3 = []

FWHMgsub1 = []
FWHMgsub2 = []
FWHMgsub3 = []
FWHMisub1 = []
FWHMisub2 = []
FWHMisub3 = []
FWHMgsci1 = []
FWHMgsci2 = []
FWHMgsci3 = []
FWHMisci1 = []
FWHMisci2 = []
FWHMisci3 = []

mag_csv = pd.DataFrame(mag_csv)
gsub1 = pd.DataFrame(gsub1)
gsub2 = pd.DataFrame(gsub2)
gsub3 = pd.DataFrame(gsub3)
isub1 = pd.DataFrame(isub1)
isub2 = pd.DataFrame(isub2)
isub3 = pd.DataFrame(isub3)
gsci1 = pd.DataFrame(gsci1)
gsci2 = pd.DataFrame(gsci2)
gsci3 = pd.DataFrame(gsci3)
isci1 = pd.DataFrame(isci1)
isci2 = pd.DataFrame(isci2)
isci3 = pd.DataFrame(isci3)

xgsub1 = pd.DataFrame(xgsub1)
xgsub2 = pd.DataFrame(xgsub2)
xgsub3 = pd.DataFrame(xgsub3)
xisub1 = pd.DataFrame(xisub1)
xisub2 = pd.DataFrame(xisub2)
xisub3 = pd.DataFrame(xisub3)
xgsci1 = pd.DataFrame(xgsci1)
xgsci2 = pd.DataFrame(xgsci2)
xgsci3 = pd.DataFrame(xgsci3)
xisci1 = pd.DataFrame(xisci1)
xisci2 = pd.DataFrame(xisci2)
xisci3 = pd.DataFrame(xisci3)

ygsub1 = pd.DataFrame(ygsub1)
ygsub2 = pd.DataFrame(ygsub2)
ygsub3 = pd.DataFrame(ygsub3)
yisub1 = pd.DataFrame(yisub1)
yisub2 = pd.DataFrame(yisub2)
yisub3 = pd.DataFrame(yisub3)
ygsci1 = pd.DataFrame(ygsci1)
ygsci2 = pd.DataFrame(ygsci2)
ygsci3 = pd.DataFrame(ygsci3)
yisci1 = pd.DataFrame(yisci1)
yisci2 = pd.DataFrame(yisci2)
yisci3 = pd.DataFrame(yisci3)

FWHMgsub1 = pd.DataFrame(FWHMgsub1)
FWHMgsub2 = pd.DataFrame(FWHMgsub2)
FWHMgsub3 = pd.DataFrame(FWHMgsub3)
FWHMisub1 = pd.DataFrame(FWHMisub1)
FWHMisub2 = pd.DataFrame(FWHMisub2)
FWHMisub3 = pd.DataFrame(FWHMisub3)
FWHMgsci1 = pd.DataFrame(FWHMgsci1)
FWHMgsci2 = pd.DataFrame(FWHMgsci2)
FWHMgsci3 = pd.DataFrame(FWHMgsci3)
FWHMisci1 = pd.DataFrame(FWHMisci1)
FWHMisci2 = pd.DataFrame(FWHMisci2)
FWHMisci3 = pd.DataFrame(FWHMisci3)

In [ ]:
for i in range(len(data1_123_matches)): #looping over all matches
    
    script_dir = os.path.dirname(__file__) #this is is preparation to make a file in the terminal 
    results_dir = os.path.join(script_dir, 'Lightcurves/')
    if not os.path.isdir(results_dir): #makes a directory if there isnt one called 'Lightcurves'
        os.makedirs(results_dir)
    
    
########################################################################################################
########################################################################################################
    #data points for the subtraction g band 
    
    
    
    time_g_sub = [8,9,10] # for the x axis of the first light curve 
    lc_g_sub = [] #dataframe for the points in the subtraction light curve
    lc_g_sub = pd.DataFrame(lc_g_sub)
    
    lc_g_sub = lc_g_sub.append(data1_123_matches.iloc[i]) #adding in the first data point
    gsub1 = gsub1.append(data1_123_matches.iloc[i,12:13]) #adding in the mag auto for the csv file
    xgsub1 = xgsub1.append(data1_123_matches.iloc[i,8:9]) #adding in the xworld for the csv file
    ygsub1 = ygsub1.append(data1_123_matches.iloc[i,9:10]) # adding in the yworld for the csv file 
    FWHMgsub1 = FWHMgsub1.append(data1_123_matches.iloc[i,23:24]) #adding in the FWHM for the csv file
    
    lc_g_sub = lc_g_sub.append(data2_123_matches.iloc[i]) # second data point
    gsub2 = gsub2.append(data2_123_matches.iloc[i,12:13])
    xgsub2 = xgsub2.append(data2_123_matches.iloc[i,8:9])
    ygsub2 = ygsub2.append(data2_123_matches.iloc[i,9:10])
    FWHMgsub2 = FWHMgsub2.append(data2_123_matches.iloc[i,23:24])
    
    lc_g_sub = lc_g_sub.append(data3_123_matches.iloc[i]) # third data point 
    gsub3 = gsub3.append(data3_123_matches.iloc[i,12:13])
    xgsub3 = xgsub3.append(data3_123_matches.iloc[i,8:9])
    ygsub3 = ygsub3.append(data3_123_matches.iloc[i,9:10])
    FWHMgsub3 = FWHMgsub3.append(data3_123_matches.iloc[i,23:24])
    
    
########################################################################################################
########################################################################################################
    # data points for the science g band
    
    
    time_g_sci = []
    lc_g_sci = [] #dataframe for the points in the science light curve
    lc_g_sci = pd.DataFrame(lc_g_sci)
    
    #matching for the science catalogue and adding data point to science light curve
    
    match1(data1_123_matches.iloc[i,8:10], df7f)
    if catalogue_match.empty == False: #if the catalogue match is not a nan, then add it to the second lightcurve
        lc_g_sci = lc_g_sci.append(catalogue_match)
        time_g_sci.append(8)
        gsci1 = gsci1.append(catalogue_match.iloc[:,12:13])
        xgsci1 = xgsci1.append(catalogue_match.iloc[:,8:9])
        ygsci1 = ygsci1.append(catalogue_match.iloc[:,9:10])
        FWHMgsci1 = FWHMgsci1.append(catalogue_match.iloc[:,23:24])
    else:
        gsci1 = gsci1.append([np.nan])
        xgsci1 = xgsci1.append([np.nan])
        ygsci1 = ygsci1.append([np.nan])
        FWHMgsci1 = FWHMgsci1.append([np.nan])
        pass
    
    match1(data1_123_matches.iloc[i,8:10], df8f)
    if catalogue_match.empty == False: #if the catalogue match is not a nan, then add it to the second lightcurve
        lc_g_sci = lc_g_sci.append(catalogue_match)
        time_g_sci.append(9)
        gsci2 = gsci2.append(catalogue_match.iloc[:,12:13])
        xgsci2 = xgsci2.append(catalogue_match.iloc[:,8:9])
        ygsci2 = ygsci2.append(catalogue_match.iloc[:,9:10])
        FWHMgsci2 = FWHMgsci2.append(catalogue_match.iloc[:,23:24])
    else:
        gsci2 = gsci2.append([np.nan])
        xgsci2 = xgsci2.append([np.nan])
        ygsci2 = ygsci2.append([np.nan])
        FWHMgsci2 = FWHMgsci2.append([np.nan])
        pass
    
    match1(data1_123_matches.iloc[i,8:10], df9f)
    if catalogue_match.empty == False: #if the catalogue match is not a nan, then add it to the second lightcurve
        lc_g_sci = lc_g_sci.append(catalogue_match)
        time_g_sci.append(10)
        gsci3 = gsci3.append(catalogue_match.iloc[:,12:13])
        xgsci3 = xgsci3.append(catalogue_match.iloc[:,8:9])
        ygsci3 = ygsci3.append(catalogue_match.iloc[:,9:10])
        FWHMgsci3 = FWHMgsci3.append(catalogue_match.iloc[:,23:24])
    else:
        gsci3 = gsci3.append([np.nan])
        xgsci3 = xgsci3.append([np.nan])
        ygsci3 = ygsci3.append([np.nan])
        FWHMgsci3 = FWHMgsci3.append([np.nan])
        pass
    
    
########################################################################################################
########################################################################################################
    # data points for the subtraction i band
    
    
    #making some dataframes for the subtraction i band 
    time_i_sub = []
    lc_i_sub = []
    lc_i_sub = pd.DataFrame(lc_i_sub)
    
    #matching for the subtraction catalogue in the i band for the subtraction light curve 
    match1(data1_123_matches.iloc[i,8:10], df4f) #finding the source in the i subtraction image (08) & appending into lightcurve
    if catalogue_match.empty == False: #if the catalogue match is not a nan, then add it to the second lightcurve
        lc_i_sub = lc_i_sub.append(catalogue_match)
        time_i_sub.append(8)
        isub1 = isub1.append(catalogue_match.iloc[:,12:13])
        xisub1 = xisub1.append(catalogue_match.iloc[:,8:9])
        yisub1 = yisub1.append(catalogue_match.iloc[:,9:10])
        FWHMisub1 = FWHMisub1.append(catalogue_match.iloc[:,23:24])
    else:
        isub1 = isub1.append([np.nan])
        xisub1 = xisub1.append([np.nan])
        yisub1 = yisub1.append([np.nan])
        FWHMisub1 = FWHMisub1.append([np.nan])
        pass 

    match1(data1_123_matches.iloc[i,8:10], df5f) #finding the source in the subtraction i image (09) etc 
    if catalogue_match.empty == False:
        lc_i_sub = lc_i_sub.append(catalogue_match)
        time_i_sub.append(9)
        isub2 = isub2.append(catalogue_match.iloc[:,12:13])
        xisub2 = xisub2.append(catalogue_match.iloc[:,8:9])
        yisub2 = yisub2.append(catalogue_match.iloc[:,9:10])
        FWHMisub2 = FWHMisub2.append(catalogue_match.iloc[:,23:24])
    else:
        isub2 = isub2.append([np.nan])
        xisub2 = xisub2.append([np.nan])
        yisub2 = yisub2.append([np.nan])
        FWHMisub2 = FWHMisub2.append([np.nan])
        pass 
        
    match1(data1_123_matches.iloc[i,8:10], df6f)
    if catalogue_match.empty == False:
        lc_i_sub = lc_i_sub.append(catalogue_match)
        time_i_sub.append(10)
        isub3 = isub3.append(catalogue_match.iloc[:,12:13])
        xisub3 = xisub3.append(catalogue_match.iloc[:,8:9])
        yisub3 = yisub3.append(catalogue_match.iloc[:,9:10])
        FWHMisub3 = FWHMisub3.append(catalogue_match.iloc[:,23:24])
    else: 
        isub3 = isub3.append([np.nan])
        xisub3 = xisub3.append([np.nan])
        yisub3 = yisub3.append([np.nan])
        FWHMisub3 = FWHMisub3.append([np.nan])
        pass

    
########################################################################################################
########################################################################################################
    # data points for the science i band 
    
    time_i_sci = []
    lc_i_sci = []
    lc_i_sci = pd.DataFrame(lc_i_sci)
    
    match1(data1_123_matches.iloc[i,8:10], df10f)
    if catalogue_match.empty == False: #if the catalogue match is not a nan, then add it to the second lightcurve
        lc_i_sci = lc_i_sci.append(catalogue_match)
        time_i_sci.append(8)
        isci1 = isci1.append(catalogue_match.iloc[:,12:13])
        xisci1 = xisci1.append(catalogue_match.iloc[:,8:9])
        yisci1 = yisci1.append(catalogue_match.iloc[:,9:10])
        FWHMisci1 = FWHMisci1.append(catalogue_match.iloc[:,23:24])
    else:
        isci1 = isci1.append([np.nan])
        xisci1 = xisci1.append([np.nan])
        yisci1 = yisci1.append([np.nan])
        FWHMisci1 = FWHMisci1.append([np.nan])
        pass
    
    match1(data1_123_matches.iloc[i,8:10], df11f)
    if catalogue_match.empty == False: #if the catalogue match is not a nan, then add it to the second lightcurve
        lc_i_sci = lc_i_sci.append(catalogue_match)
        time_i_sci.append(9)
        isci2 = isci2.append(catalogue_match.iloc[:,12:13])
        xisci2 = xisci2.append(catalogue_match.iloc[:,8:9])
        yisci2 = yisci2.append(catalogue_match.iloc[:,9:10])
        FWHMisci2 = FWHMisci2.append(catalogue_match.iloc[:,23:24])
    else:
        isci2 = isci2.append([np.nan])
        xisci2 = xisci2.append([np.nan])
        yisci2 = yisci2.append([np.nan])
        FWHMisci2 = FWHMisci2.append([np.nan])
        pass
    
    match1(data1_123_matches.iloc[i,8:10], df12f)
    if catalogue_match.empty == False: #if the catalogue match is not a nan, then add it to the second lightcurve
        lc_i_sci = lc_i_sci.append(catalogue_match)
        time_i_sci.append(10)
        isci3 = isci3.append(catalogue_match.iloc[:,12:13])
        xisci3 = xisci3.append(catalogue_match.iloc[:,8:9])
        yisci3 = yisci3.append(catalogue_match.iloc[:,9:10])
        FWHMisci3 = FWHMisci3.append(catalogue_match.iloc[:,23:24])
    else:
        isci3 = isci3.append([np.nan])
        xisci3 = xisci3.append([np.nan])
        yisci3 = yisci3.append([np.nan])
        FWHMisci3 = FWHMisci3.append([np.nan])
        pass

########################################################################################################
########################################################################################################

    # making the points for the 'colour' in the subtraction image

    if bool(time_i_sub) == True & bool(time_g_sub) == True: #if time_i_sub is not empty, execute the next steps, otherwise pass
        mag_g_sub = lc_g_sub['MAG_AUTO']
        mag_i_sub = lc_i_sub['MAG_AUTO']
        
        mag_g_subm = mag_g_sub.reset_index()
        mag_i_subm = mag_i_sub.reset_index()
        
        mag_g_subm = mag_g_subm['MAG_AUTO']
        mag_i_subm = mag_i_subm['MAG_AUTO']
        
        mag_diff_sub = mag_g_subm - mag_i_subm
    else: 
        pass
    
########################################################################################################
########################################################################################################
    
    # making the points for the 'colour' in the science image

    if bool(time_i_sci) == True & bool(time_g_sci) == True: #if time_i_sub is not empty, execute the next steps, otherwise pass
        mag_g_sci = lc_g_sci['MAG_AUTO']
        mag_i_sci = lc_i_sci['MAG_AUTO']
        
        mag_g_scim = mag_g_sci.reset_index()
        mag_i_scim = mag_i_sci.reset_index()
        
        mag_g_scim = mag_g_scim['MAG_AUTO']
        mag_i_scim = mag_i_scim['MAG_AUTO']
        
        mag_diff_sci = mag_g_scim - mag_i_scim
    else: 
        pass

########################################################################################################
########################################################################################################

    # plotting the subtraction light curve  
    
    RA_sub = lc_g_sub['X_WORLD'] #these will be for the title of the light curves 
    Dec_sub = lc_g_sub['Y_WORLD']
    
    fig, ax = plt.subplots(2, figsize = (10,14), sharey = False) #light curve plotting parameters 
    ax0 = plt.subplot(2,1,1)
    plt.title('RA: '+ str(RA_sub.iloc[0]) + ' Dec: '+  str(Dec_sub.iloc[0])) #RA and Dec of the first data point in te dataframe in this case it is the RA and Dec of the subtraction point in 08
    ax0.set_xlabel('Date')
    ax0.set_ylabel('Magnitude in subtraction image', color = 'black')
    plt.gca().invert_yaxis()
    ax0.plot(time_g_sub, mag_g_sub, color ='g', marker = 'o', label = 'g')
    
    
    if bool(time_i_sub) == True: #if there are any points for the i band, plot them down, otherwise do nothing
        ax0.plot(time_i_sub, mag_i_sub, color = 'grey', marker = 'o', label = 'i')
        plt.legend()

        ax1 = ax0.twinx()  # initiates a second axes that shares the same x-axis for the 'colour' plot 
        ax1.set_ylabel('g - i', color = 'r')  # we already handled the x-label with ax1
        ax1.tick_params(axis='y', labelcolor = 'r')
        ax1.plot(time_g_sub, mag_diff_sub, color = 'r', marker = 'x', label = 'g-i')
        
    else:
        plt.legend()
        pass

#     plt.locator_params(axis = 'y', nbins = 10) #limiting the x-axis and y-axis
#     plt.locator_params(axis='x', nbins=10)

########################################################################################################
########################################################################################################
    
    # plotting the science light curve
   
#     RA_sci = lc_g_sci['X_WORLD'] #these will be for the title of the light curves 
#     Dec_sci = lc_g_sci['Y_WORLD']
    ax2 = plt.subplot(2,1,2)
    
    if bool(time_g_sci) == True:# if there are any points in the science g band, plot them down 
#         plt.title('RA: '+ str(RA_sci.iloc[0]) + ' Dec: '+  str(Dec_sci.iloc[0]))
        ax2.set_xlabel('Date')
        ax2.set_ylabel('Magnitude in science image', color = 'black')
        try:
            ax2.plot(time_g_sci, mag_g_sci, color ='g', marker = 'o', label = 'g')
            
        except(ValueError): 
            pass
            
        
    else: 
        pass
    
    if bool(time_i_sci) == True: #if there are any points int he science i badn, plot them down
#         plt.title('RA: '+ str(RA_sci.iloc[0]) + ' Dec: '+  str(Dec_sci.iloc[0]))
        ax2.set_xlabel('Date')
        ax2.set_ylabel('Magnitude in science image', color = 'black')
        ax2.invert_yaxis()
        try:
            ax2.plot(time_i_sci, mag_i_sci, color = 'grey', marker = 'o', label = 'i')
        except(ValueError):
            pass
        
        plt.legend()
        
        ax3 = ax2.twinx()  # initiates a second axes that shares the same x-axis for the 'colour' plot 
        ax3.set_ylabel('g - i', color = 'r')  # we already handled the x-label with ax1
        ax3.tick_params(axis='y', labelcolor = 'r')
        ax3.invert_yaxis()
        try: 
            ax3.plot(time_g_sci, mag_diff_sci, color = 'r', marker = 'x', label = 'g-i')
        except(ValueError):
            pass
        
    else:
        plt.legend()
        pass
    
#     plt.locator_params(axis = 'y', nbins = 10) #limiting the x-axis and y-axis
#     plt.locator_params(axis='x', nbins=10)
    
    plt.savefig(results_dir + 'CAND_'+ str(i) + '_RA: '+ str(RA_sub.iloc[0]) + '_Dec: '+  str(Dec_sub.iloc[0]) + 'Lightcurve' + '.png', format = 'png') #saving the light curve as an image
    # the results_dir tells python to save the file in that specific directory
    
    plt.close() #closing plots to save memory etc 
    
    if i % 10 == 0: #printing progress every 10 iterations
        print('i = {}'.format(i) + '/' + str(len(data1_123_matches)))
        
########################################################################################################
########################################################################################################

# Exporting the data onto a csv file 
        
gsub1 = gsub1.reset_index()
gsub2 = gsub2.reset_index()
gsub3 = gsub3.reset_index()
isub1 = isub1.reset_index()
isub2 = isub2.reset_index()
isub3 = isub3.reset_index()
gsci1 = gsci1.reset_index()
gsci2 = gsci2.reset_index()
gsci3 = gsci3.reset_index()
isci1 = isci1.reset_index()
isci2 = isci2.reset_index()
isci3 = isci3.reset_index()

xgsub1 = xgsub1.reset_index()
xgsub2 = xgsub2.reset_index()
xgsub3 = xgsub3.reset_index()
ygsub1 = ygsub1.reset_index()
ygsub2 = ygsub2.reset_index()
ygsub3 = ygsub3.reset_index()
xgsci1 = xgsci1.reset_index()
xgsci2 = xgsci2.reset_index()
xgsci3 = xgsci3.reset_index()
ygsci1 = ygsci1.reset_index()
ygsci2 = ygsci2.reset_index()
ygsci3 = ygsci3.reset_index()

xisub1 = xisub1.reset_index()
xisub2 = xisub2.reset_index()
xisub3 = xisub3.reset_index()
yisub1 = yisub1.reset_index()
yisub2 = yisub2.reset_index()
yisub3 = yisub3.reset_index()
xisci1 = xisci1.reset_index()
xisci2 = xisci2.reset_index()
xisci3 = xisci3.reset_index()
yisci1 = yisci1.reset_index()
yisci2 = yisci2.reset_index()
yisci3 = yisci3.reset_index()

FWHMgsub1 = FWHMgsub1.reset_index()
FWHMgsub2 = FWHMgsub2.reset_index()
FWHMgsub3 = FWHMgsub3.reset_index()
FWHMisub1 = FWHMisub1.reset_index()
FWHMisub2 = FWHMisub2.reset_index()
FWHMisub3 = FWHMisub3.reset_index()
FWHMgsci1 = FWHMgsci1.reset_index()
FWHMgsci2 = FWHMgsci2.reset_index()
FWHMgsci3 = FWHMgsci3.reset_index()
FWHMisci1 = FWHMisci1.reset_index()
FWHMisci2 = FWHMisci2.reset_index()
FWHMisci3 = FWHMisci3.reset_index()

mag_csv = mag_csv.assign(gsub1 = gsub1['MAG_AUTO'] , gsub2 = gsub2['MAG_AUTO'], gsub3 = gsub3['MAG_AUTO'], 
                        isub1 = isub1['MAG_AUTO'], isub2 = isub2['MAG_AUTO'], isub3 = isub3['MAG_AUTO'],
                        gsci1 = gsci1['MAG_AUTO'] , gsci2 = gsci2['MAG_AUTO'], gsci3 = gsci3['MAG_AUTO'],
                        isci1 = isci1['MAG_AUTO'] , isci2 = isci2['MAG_AUTO'], isci3 = isci3['MAG_AUTO'],
                         
                        xgsub1 = xgsub1['X_WORLD'], xgsub2 = xgsub2['X_WORLD'], xgsub3 = xgsub3['X_WORLD'],
                        ygsub1 = ygsub1['Y_WORLD'], ygsub2 = ygsub2['Y_WORLD'], ygsub3 = ygsub3['Y_WORLD'],
                        xgsci1 = xgsci1['X_WORLD'], xgsci2 = xgsci2['X_WORLD'], xgsci3 = xgsci3['X_WORLD'],
                        ygsci1 = ygsci1['Y_WORLD'], ygsci2 = ygsci2['Y_WORLD'], ygsci3 = ygsci3['Y_WORLD'],
                         
                        xisub1 = xisub1['X_WORLD'], xisub2 = xisub2['X_WORLD'], xisub3 = xisub3['X_WORLD'],
                        yisub1 = yisub1['Y_WORLD'], yisub2 = yisub2['Y_WORLD'], yisub3 = yisub3['Y_WORLD'],
                        xisci1 = xisci1['X_WORLD'], xisci2 = xisci2['X_WORLD'], xisci3 = xisci3['X_WORLD'],
                        yisci1 = yisci1['Y_WORLD'], yisci2 = yisci2['Y_WORLD'], yisci3 = yisci3['Y_WORLD'],
                         
                        FWHMgsub1 = FWHMgsub1['FWHM_WORLD'], FWHMgsub2 = FWHMgsub2['FWHM_WORLD'], FWHMgsub3 = FWHMgsub3['FWHM_WORLD'],
                        FWHMisub1 = FWHMisub1['FWHM_WORLD'], FWHMisub2 = FWHMisub2['FWHM_WORLD'], FWHMisub3 = FWHMisub3['FWHM_WORLD'],
                        FWHMgsci1 = FWHMgsci1['FWHM_WORLD'], FWHMgsci2 = FWHMgsci2['FWHM_WORLD'], FWHMgsci3 = FWHMgsci3['FWHM_WORLD'],
                        FWHMisci1 = FWHMisci1['FWHM_WORLD'], FWHMisci2 = FWHMisci2['FWHM_WORLD'], FWHMisci3 = FWHMisci3['FWHM_WORLD'])
                        


mag_csv.to_csv(results_dir + 'Sources.csv') #writing out all of the magnitudes into a csv file